In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于在线预测的AutoML图像物体检测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_object_detection_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_object_detection_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Python的Vertex客户端库来创建图像对象检测模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行在线预测。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中“沙拉”类别的[OpenImages数据集](https://www.tensorflow.org/datasets/catalog/overview)。这个数据集不需要任何特征工程。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测图像中五类物品中的沙拉、海鲜、番茄、烘焙食品或奶酪的边界框位置和相应类型。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML图像目标检测模型，并部署到在线预测。您也可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个用于提供预测的`serving Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 费用

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户库和Google *云存储*，您需要重新启动笔记本内核以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有可能，确保你在 GPU 运行时中运行这个 notebook。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置你的谷歌云项目

**无论你的 notebook 环境如何，以下步骤都是必须的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你首次创建一个帐户时，你将获得 $300 的免费信用额度以用于计算/存储费用。

2. [确保你的项目已启用计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入你的项目 ID。然后运行这个单元格，确保 Cloud SDK 对这个 notebook 中的所有命令使用正确的项目。

**注意**: Jupyter 会将以 `!` 开头的行作为 shell 命令运行，并将以 `$` 开头的 Python 变量插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于本笔记本的其余操作。 以下是支持Vertex的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来训练Vertex。并非所有地区都支持所有Vertex服务。有关每个地区的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在一个实时教程会话中，您可能会使用一个共享的测试账户或项目。为避免用户在创建的资源上发生重名冲突，您需要为每个实例会话创建一个时间戳，并附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Google Cloud 笔记本**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并按提示进行身份验证，通过 oAuth。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，然后选择**Vertex 管理员**。在过滤框中键入“Storage Object Admin”，然后选择**Storage Object Admin**。

点击创建。一个包含您密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中，将服务帐户密钥的路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入 Vertex 客户端库

将 Vertex 客户端库导入到我们的 Python 环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

### Vertex常数

为Vertex设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、管道和终端服务的Vertex API服务端点。
- `PARENT`：用于数据集、模型、作业、管道和终端资源的Vertex位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常量

设置专门针对AutoML数据集和训练的常量：

- 数据集模式：告诉`数据集`资源服务它是什么类型的数据集。
- 数据标记（注释）模式：告诉`数据集`资源服务数据是如何标记（注释）的。
- 数据集训练模式：告诉`管道`资源服务要为哪个任务（例如分类）训练模型。

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_bounding_box_io_format_1.0.0.yaml"
# Image Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml"

#教程

现在您已经准备好开始创建自己的AutoML图像目标检测模型了。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本），您将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请预先设置它们。

- 数据集服务 用于 `Dataset` 资源。
- 模型服务 用于 `Model` 资源。
- 流水线服务 用于训练。
- 终端服务 用于部署。
- 预测服务 用于服务端。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，培训模型的第一步是创建一个托管的数据集实例，然后将标记数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 使用以下参数创建一个Vertex `Dataset`资源(`aip.Dataset`)：
 - `display_name`：您选择的可读名称。
 - `metadata_schema_uri`：数据集类型的架构。
3. 调用客户端数据集服务方法`create_dataset`，使用以下参数：
 - `parent`：您的`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的异步调用的方式。虽然这一步通常很快，但在项目中首次使用时，由于需要进行配置，可能会有更长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源），或者通过调用操作方法来取消操作：

| 方法        | 描述            |
| ----------- | --------------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。     |
| running()   | 返回操作是否仍在运行的True/False。        |
| done()      | 返回操作是否已完成的True/False。           |
| canceled()  | 返回操作是否已取消的True/False。           |
| cancel()    | 取消操作（可能需要最多30秒）。           |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("salads-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的`Dataset`资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

图像的 Vertex `Dataset` 资源对您的数据有一些要求：

- 图像必须存储在 Cloud 存储桶中。
- 每个图像文件必须是图像格式（PNG，JPEG，BMP，...）。
- 在您的 Cloud 存储桶中必须有一个包含每个图像路径和标签的索引文件。
- 索引文件必须是 CSV 或 JSONL 格式。

CSV索引文件对于图像对象检测有以下要求：

- 没有标题。
- 第一列是图像的Cloud Storage路径。
- 第二列是标签。
- 第三/四列是边界框的左上角。坐标已归一化，介于0和1之间。
- 第五/六/七列不被使用，应该为0。
- 第八/九列是边界框的右下角。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的Salads数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数来计算示例的数量（`wc -l`），然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的Vertex Dataset资源中。使用这个辅助函数`import_data`来导入数据。该函数执行以下操作：

- 使用`Dataset`客户端。
- 调用客户端方法`import_data`，并使用以下参数：
  - `name`：您给`Dataset`资源的可读名称（例如，沙拉）。
  - `import_configs`：导入配置。

- `import_configs`：包含一个字典的Python列表，其中包含键/值条目：
  - `gcs_sources`：一个包含指向一个或多个索引文件路径的URI列表。
  - `import_schema_uri`：标识标签类型的模式。

`import_data()`方法返回一个长时间运行的`operation`对象。这将花费几分钟的时间完成。如果您正在参加实时教程，现在是一个提问或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

训练模型

现在，使用您的 Vertex `Dataset` 资源来训练一个 AutoML 图像目标检测模型。为了训练模型，请执行以下步骤：

1. 为 `Dataset` 资源创建 Vertex 训练流水线。
2. 执行流水线开始训练。

### 创建培训流水线

您可能会问，我们用流水线做什么？一般情况下，当工作（如培训）有多个步骤时，通常按顺序进行：执行步骤A，执行步骤B等等。通过将这些步骤放入流水线中，我们可以获得以下好处：

1. 可以重复用于后续的培训工作。
2. 可以容器化并作为批处理任务运行。
3. 可以分布式执行。
4. 所有步骤与相同的流水线作业关联，以跟踪进度。

使用这个帮助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：流水线作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标记（注释）培训架构。
- `task`：描述培训作业要求的字典。

这个帮助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，它接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：流水线培训作业的全部规范。

现在让我们更深入地了解构建 `training_pipeline` 规范的*最低*要求：

- `display_name`：流水线作业的人类可读名称。
- `training_task_definition`：数据集标记（注释）培训架构。
- `training_task_inputs`：描述培训作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅为 Vertex 数据集标识符（非完全限定）--这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，则用于培训、测试和验证的数据集百分比。否则，百分比将自动由 AutoML 选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，需要传入一个 Google protobuf 结构体的 `task` 字段，该结构体与 Python 字典非常相似，可以使用 `json_format.ParseDict` 方法进行转换。

您需要指定的最小字段包括：

- `budget_milli_node_hours`：用于训练模型的最大时间预算（计费时间），其中 1000 = 1 小时。对于图像对象检测，预算必须至少为 20 小时。
- `model_type`：部署模型的类型：
  - `CLOUD_HIGH_ACCURACY_1`：用于部署到 Google Cloud 并优化准确性。
  - `CLOUD_LOW_LATENCY_1`：用于部署到 Google Cloud 并优化延迟（响应时间）。
  - `MOBILE_TF_HIGH_ACCURACY_1`：用于部署到边缘并优化准确性。
  - `MOBILE_TF_LOW_LATENCY_1`：用于部署到边缘并优化延迟（响应时间）。
  - `MOBILE_TF_VERSATILE_1`：用于部署到边缘并在延迟和准确性之间进行权衡。
- `disable_early_stopping`：True/False，用于指示 AutoML 是否根据自身判断提前停止训练或继续训练至整个预算。

最后，通过调用辅助函数 `create_pipeline` 来创建管道，该函数返回一个训练管道对象的实例。

In [ ]:
PIPE_NAME = "salads_pipe-" + TIMESTAMP
MODEL_NAME = "salads_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "budget_milli_node_hours": 20000,
        "model_type": "CLOUD_HIGH_ACCURACY_1",
        "disable_early_stopping": False,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关训练管道的信息

现在仅获取此训练管道实例的管道信息。通过调用作业客户端服务的`get_training_pipeline`方法，使用以下参数来获取此作业的作业信息:

- `name`: Vertex的完全限定管道标识符。

当模型训练完成时，管道状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练上述模型可能需要多达60分钟的时间。

一旦您的模型训练完成，您可以通过从“end_time”减去“start_time”来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的Vertex模型资源标识符，这是管道服务为其分配的。您可以从返回的管道实例中获取此信息，作为字段“model_to_deploy.name”。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在您的模型已经训练完毕，您可以获取一些关于您的模型的信息。

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被设置为测试（保留）数据，由管道服务用来评估模型。

### 列出所有切片的评估

使用此辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model`资源的 Vertex 完全限定模型标识符。

此辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估--您可能只有一个，然后打印出评估中每个指标的所有关键名称，并对一小组指标（`evaluatedBoundingBoxCount`和`boundingBoxMeanAveragePrecision`）打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("evaluatedBoundingBoxCount", metrics["evaluatedBoundingBoxCount"])
        print(
            "boundingBoxMeanAveragePrecision",
            metrics["boundingBoxMeanAveragePrecision"],
        )

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 部署`Model`资源

现在部署使用AutoML创建的训练后的Vertex `Model`资源。这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以提供预测的终端节点，使用以下参数：

- `display_name`：`Endpoint`资源的可读名称。

该辅助函数使用终端节点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：`Endpoint`资源的可读名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供服务的`Endpoint`资源提供资源。您可以调用`response.result()`，这是一个同步调用，并在终端节点资源准备就绪时返回。该辅助函数返回`Endpoint`资源的Vertex完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "salads_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的`Endpoint`资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例的扩展

对于处理您的在线预测请求，您有几种计算实例扩展的选择：

- 单个实例: 在单个计算实例上处理在线预测请求。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展: 在您手动指定的固定数量的计算实例上分配在线预测请求。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数量。当模型首次部署到实例时，将预先配置固定数量的计算实例，并且在线预测请求将均匀分配到它们之间。

- 自动扩展: 在一可扩展数量的计算实例上分割在线预测请求。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时需要预配和取消预配的数量，并将最大（`MAX_NODES`）数量设置为根据负载条件需要进行预配的计算实例数量。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在后续的部署请求中可以使用。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

请使用这个辅助函数 `deploy_model` 来将您为提供预测创建的 `Model` 资源部署到 `Endpoint` 资源中，使用以下参数：

- `model`：要从训练管道中上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将模型部署到的 Vertex 完全限定端点标识符。

辅助函数调用 `Endpoint` 客户端服务的 `deploy_model` 方法，该方法需要以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`：用于部署模型的要求规范。
- `traffic_split`：流量在端点中流向此模型的百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则请指定为 **{ "0": 100 }**，其中 "0" 指的是正在上传的模型，100 表示100% 的流量。
   - 如果在端点上有现有的模型，流量将被分割到这些模型，请使用 `model_id` 来指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是待部署端点上现有模型的模型 ID。百分比必须总和为100。

现在让我们更深入地了解 `deployed_model` 参数。该参数被指定为一个具有最小所需字段的 Python 字典：

- `model`：要部署的（上传）模型的 Vertex 完全限定模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：这将禁用容器事件的记录，例如执行失败（默认情况下为启用容器记录）。通常在部署程序调试时启用容器记录，然后在部署到生产环境时禁用。
- `automatic_resources`：这指的是多少个冗余计算实例（副本）。在这个示例中，我们将其设置为一个（不复制）。

#### 流量分配

现在让我们更深入地了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。起初可能有点令人困惑。让我解释一下，您可以将模型的多个实例部署到一个端点，然后设置每个实例接收多少流量（百分比）。

为什么要这样做？也许您已经在生产环境中部署了之前的版本 - 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在真正部署到生产环境之前，您不能确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但它只占有10% 的流量。这样，您可以在不中断大多数用户的情况下监视其表现，直到作出最终决定。

#### 响应

此方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，这将阻塞直到模型部署完成。如果这是首次将模型部署到端点，可能需要额外几分钟完成资源的配置。

In [ ]:
DEPLOYED_NAME = "salads_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "automatic_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

## 发起在线预测请求

现在请对部署的模型进行在线预测。

获取测试项目

您将会从数据集中随机选择一个示例作为测试项目。不必担心这个示例很可能在训练模型时被使用过 -- 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n1
cols = str(test_items[0]).split(",")
if len(cols) == 11:
    test_item = str(cols[1])
    test_label = str(cols[2])
else:
    test_item = str(cols[0])
    test_label = str(cols[1])

print(test_item, test_label)

### 进行预测

现在您有一个测试项目。使用以下辅助函数`predict_item`，其接受以下参数：

- `filename`：测试项目的云存储路径。
- `endpoint`：部署`Model`资源的`Endpoint`资源的Vertex完全限定标识符。
- `parameters_dict`：用于提供预测结果的额外过滤参数。

该函数调用预测客户端服务的`predict`方法，使用以下参数：

- `endpoint`：部署`Model`资源的`Endpoint`资源的Vertex完全限定标识符。
- `instances`：要预测的实例（编码图像）列表。
- `parameters`：用于提供的额外参数。
  - `confidence_threshold`：返回预测的阈值。必须介于0和1之间。
  - `max_predictions`：要返回的每个对象的最大预测数，按信心排序。

您可能会问，`confidence_threshold`如何影响模型的准确性？该阈值不会改变准确性。它改变的是召回率和精准率。

   - 精准率：精准率越高，预测的准确性更高，但返回的预测结果更少。增加置信度阈值将增加精准率。
   - 召回率：召回率越高，结果中返回的正确预测更有可能，但会返回更多的带有不正确预测的预测结果。降低置信度阈值将增加召回率。

在这个示例中，您将预测精准率。您将把置信度阈值设置为0.5，对于一个对象的最大预测次数为两次。由于所有类别的置信度值必须加起来等于1，只有两种可能的结果：

   1. 出现平局，都为0.5，并返回两个预测。
   2. 一个值大于0.5，其余所有值小于0.5，并返回一个预测。

#### 请求

由于在这个示例中，您的测试项目在一个云存储存储桶中，您将打开并读取图像内容，使用`tf.io.gfile.Gfile()`。为了将测试数据传递给预测服务，我们将字节编码为base 64——这样在通过互联网传输时，可以确保二进制数据不被修改。

每个实例的格式为：

    { 'content': { 'b64': [经过base64编码的字节] } }

由于`predict()`方法可以接受多个项目（实例），您将我们的单个测试项目作为一个测试项目列表发送。最后一步是将实例列表打包成Google的protobuf格式——这是我们传递给`predict()`方法的内容。

#### 响应

`response`对象返回一个列表，列表中的每个元素对应请求中的相应图像。您将在每个预测的输出中看到：

- `confidences`：预测的置信度水平。
- `displayNames`：预测的标签。
- `bboxes`：标签的边界框。

In [ ]:
import base64

import tensorflow as tf


def predict_item(filename, endpoint, parameters_dict):

    parameters = json_format.ParseDict(parameters_dict, Value())
    with tf.io.gfile.GFile(filename, "rb") as f:
        content = f.read()
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))


predict_item(test_item, endpoint_id, {"confidenceThreshold": 0.5, "maxPredictions": 2})

取消部署 `Model` 资源

现在取消部署您的 `Model` 资源从服务 `Endpoint` 资源中。使用这个助手函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将 `Model` 资源部署到时由端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 Vertex 完全限定标识符的 `Endpoint` 资源。

此函数调用端点客户端服务的方法 `undeploy_model`，带有以下参数：

- `deployed_model_id`：在部署 `Model` 资源时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符的 `Endpoint` 资源。
- `traffic_split`：如何在 `Endpoint` 资源上的其余部署模型间分割流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空 {} 来简单地将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理工作

要清理本项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批量作业
- 自定义作业
- 超参数调整作业
- Cloud Storage 存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME